In [77]:
%env MODEL_NAME=densenet161
%env PROJECT_NAME=tsaikevin-inference
%env ENDPOINT=https://alpha-ml.googleapis.com/v1

env: MODEL_NAME=densenet161
env: PROJECT_NAME=tsaikevin-inference
env: ENDPOINT=https://alpha-ml.googleapis.com/v1


In [2]:
!gcloud config set project tsaikevin-inference

Updated property [core/project].


In [78]:
!curl -X POST -v -k -H "Content-Type: application/json" -d "{'name': '"${MODEL_NAME}"'}" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "${ENDPOINT}/projects/${PROJECT_NAME}/models/"

Note: Unnecessary use of -X or --request, POST is already inferred.
*   Trying 216.58.194.202:443...
* TCP_NODELAY set
* Connected to alpha-ml.googleapis.com (216.58.194.202) port 443 (#0)
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*   CAfile: /opt/anaconda3/ssl/cacert.pem
  CApath: none
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
* TLSv1.3 (IN), TLS handshake, Server hello (2):
* TLSv1.3 (IN), TLS handshake, Encrypted Extensions (8):
* TLSv1.3 (IN), TLS handshake, Certificate (11):
* TLSv1.3 (IN), TLS handshake, CERT verify (15):
* TLSv1.3 (IN), TLS handshake, Finished (20):
* TLSv1.3 (OUT), TLS change cipher, Change cipher spec (1):
* TLSv1.3 (OUT), TLS handshake, Finished (20):
* SSL connection using TLSv1.3 / TLS_AES_256_GCM_SHA384
* ALPN, server accepted to use http/1.1
* Server certificate:
*  subject: C=US; ST=California; L=Mountain View; O=Google LLC; CN=upload.video.google.com
*  start date: May  5 08:30:35 2020 GMT
*  expire date: Jul 28 

In [116]:
%env VERSION_NAME=v005

import json
import os

torchserve_version = {
  "name": "v005",
  "deployment_uri": "gs://tsaikevin-inference-server-repository/torchserve",
  "container": {
    "image": "gcr.io/tsaikevin-inference/caip-torchserve:latest",
    "args": [
    ],
    "env": [
    ], 
    "ports": [
      { "containerPort": 8080 }
    ]
  },
  "routes": {
    "predict": "/predictions/densenet161",
    "health": "/ping"
  },
  "machine_type": "n1-standard-4"
}
with open("torchserve_version.json", "w") as f: 
  json.dump(torchserve_version, f)

env: VERSION_NAME=v005


In [117]:
!curl -X POST -v -k -H "Content-Type: application/json" -d @torchserve_version.json \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "${ENDPOINT}/projects/${PROJECT_NAME}/models/${MODEL_NAME}/versions"

Note: Unnecessary use of -X or --request, POST is already inferred.
*   Trying 172.217.6.74:443...
* TCP_NODELAY set
* Connected to alpha-ml.googleapis.com (172.217.6.74) port 443 (#0)
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*   CAfile: /opt/anaconda3/ssl/cacert.pem
  CApath: none
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
* TLSv1.3 (IN), TLS handshake, Server hello (2):
* TLSv1.3 (IN), TLS handshake, Encrypted Extensions (8):
* TLSv1.3 (IN), TLS handshake, Certificate (11):
* TLSv1.3 (IN), TLS handshake, CERT verify (15):
* TLSv1.3 (IN), TLS handshake, Finished (20):
* TLSv1.3 (OUT), TLS change cipher, Change cipher spec (1):
* TLSv1.3 (OUT), TLS handshake, Finished (20):
* SSL connection using TLSv1.3 / TLS_AES_256_GCM_SHA384
* ALPN, server accepted to use http/1.1
* Server certificate:
*  subject: C=US; ST=California; L=Mountain View; O=Google LLC; CN=upload.video.google.com
*  start date: May  5 08:30:35 2020 GMT
*  expire date: Jul 28 08:3

In [120]:
!curl -X GET -k -H "Content-Type: application/json" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "${ENDPOINT}/projects/${PROJECT_NAME}/models/${MODEL_NAME}/versions/${VERSION_NAME}"

{
  "name": "projects/tsaikevin-inference/models/densenet161/versions/v005",
  "isDefault": true,
  "deploymentUri": "gs://tsaikevin-inference-server-repository/torchserve",
  "createTime": "2020-05-28T09:52:03Z",
  "state": "READY",
  "etag": "vhi3slq/VM0=",
  "machineType": "n1-standard-4",
  "container": {
    "image": "gcr.io/tsaikevin-inference/caip-torchserve:latest",
    "ports": [
      {
        "containerPort": 8080
      }
    ]
  },
  "routes": {
    "predict": "/predictions/densenet161",
    "health": "/ping"
  }
}


In [121]:
!curl -O https://s3.amazonaws.com/model-server/inputs/kitten.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  108k  100  108k    0     0   148k      0 --:--:-- --:--:-- --:--:--  148k


In [122]:
!curl -X POST ${ENDPOINT}/projects/${PROJECT_NAME}/models/${MODEL_NAME}/versions/${VERSION_NAME}:predict \
    -k -H "Content-Type: application/octet-stream" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    --data-binary "@kitten.jpg"

[
  {
    "tiger_cat": 0.46933549642562866
  },
  {
    "tabby": 0.4633878469467163
  },
  {
    "Egyptian_cat": 0.06456148624420166
  },
  {
    "lynx": 0.0012828214094042778
  },
  {
    "plastic_bag": 0.00023323034110944718
  }
]